<a href="https://colab.research.google.com/github/JorgeAccardi/cienciadedatos/blob/main/Analisis_de_Inclinometros_(GKN)_datos_crudos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL - Procesamiento de datos de inclinómetros
## Paso 1: Carga y concatenación de datos crudos

Este notebook procesa las lecturas de inclinómetros almacenadas en archivos `.gkn`,
extrayendo la información relevante y consolidándola en un solo `DataFrame`.

### Objetivos:
1. Leer múltiples archivos `.gkn` y extraer datos clave.
2. Unificar las lecturas en un `DataFrame`, incorporando fecha, hora e identificadores del proyecto.
3. Guardar los datos procesados en un archivo `.csv`.

---

In [ ]:
import pandas as pd
import glob
import os
from datetime import datetime

carpeta_lecturas = '/content/'
archivos_gkn = sorted(glob.glob(os.path.join(carpeta_lecturas, '*.gkn')))
dfs = []

def extraer_metadatos_gkn(path_archivo):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    fecha, hora, hole_no, project, probe_no = None, None, None, None, None
    for line in lines:
        if line.startswith("DATE"):
            fecha_str = line.split(":", 1)[1].strip()
            fecha = datetime.strptime(fecha_str, "%d/%m/%y")
        elif line.startswith("TIME"):
            hora_str = line.split(":", 1)[1].strip()
            hora = datetime.strptime(hora_str, "%H:%M:%S").time()
        elif line.startswith("HOLE NO"):
            hole_no = line.split(":", 1)[1].strip()
        elif line.startswith("PROJECT"):
            project = line.split(":", 1)[1].strip()
        elif line.startswith("PROBE NO"):
            probe_no = line.split(":", 1)[1].strip()
    if fecha and hora:
        fecha_hora = datetime.combine(fecha.date(), hora)
    elif fecha:
        fecha_hora = fecha
    else:
        fecha_hora = None
    return fecha_hora, hole_no, project, probe_no

def leer_gkn_a_dataframe(path_archivo, fecha_hora, hole_no, project, probe_no):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    idx_tabla = None
    for idx, line in enumerate(lines):
        if line.strip().startswith("FLEVEL"):
            idx_tabla = idx
            break
    if idx_tabla is not None:
        # Preprocesar: eliminar comas de cada línea de la tabla antes de leer con pandas
        tabla_limpia = []
        for line in lines[idx_tabla:]:
            tabla_limpia.append(line.replace(",", ""))
        from io import StringIO
        tabla_str = "".join(tabla_limpia)
        df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
        # Solo inclinometro, no hole_no
        df.insert(0, 'inclinometro', hole_no)
        df.insert(1, 'fecha_hora', fecha_hora)
        df['project'] = project
        df['probe_no'] = probe_no
        return df
    return None

for archivo in archivos_gkn:
    fecha_hora, hole_no, project, probe_no = extraer_metadatos_gkn(archivo)
    df = leer_gkn_a_dataframe(archivo, fecha_hora, hole_no, project, probe_no)
    if df is not None:
        dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)
output_csv = '/content/lecturas_inclinometro_unificado.csv'
df_total.to_csv(output_csv, index=False)

print(f"Archivo CSV guardado en: {output_csv}")
df_total.head()

<ipython-input-60-8b1dc62d8573>:50: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-60-8b1dc62d8573>:50: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-60-8b1dc62d8573>:50: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-60-8b1dc62d8573>:50: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_wh

Archivo CSV guardado en: /content/lecturas_inclinometro_unificado.csv


,inclinometro,fecha_hora,FLEVEL,A+,A-,B+,B-,project,probe_no
0,MD-IN1-150,2022-06-29 10:44:28,55.5,970,-994,-14,105,MD,Probe-93
1,MD-IN1-150,2022-06-29 10:44:28,55.0,968,-983,-15,110,MD,Probe-93
2,MD-IN1-150,2022-06-29 10:44:28,54.5,960,-976,-14,110,MD,Probe-93
3,MD-IN1-150,2022-06-29 10:44:28,54.0,975,-992,-7,98,MD,Probe-93
4,MD-IN1-150,2022-06-29 10:44:28,53.5,982,-996,5,90,MD,Probe-93


In [ ]:
import pandas as pd
import glob
import os
from datetime import datetime

carpeta_lecturas = '/content/'
archivos_gkn = sorted(glob.glob(os.path.join(carpeta_lecturas, '*.gkn')))
dfs = []

def extraer_metadatos_gkn(path_archivo):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    fecha, hora, hole_no, project, probe_no = None, None, None, None, None
    for line in lines:
        if line.startswith("DATE"):
            fecha_str = line.split(":", 1)[1].strip()
            fecha = datetime.strptime(fecha_str, "%d/%m/%y")
        elif line.startswith("TIME"):
            hora_str = line.split(":", 1)[1].strip()
            hora = datetime.strptime(hora_str, "%H:%M:%S").time()
        elif line.startswith("HOLE NO"):
            hole_no = line.split(":", 1)[1].strip()
        elif line.startswith("PROJECT"):
            project = line.split(":", 1)[1].strip()
        elif line.startswith("PROBE NO"):
            probe_no = line.split(":", 1)[1].strip()
    if fecha and hora:
        fecha_hora = datetime.combine(fecha.date(), hora)
    elif fecha:
        fecha_hora = fecha
    else:
        fecha_hora = None
    return fecha_hora, hole_no, project, probe_no

def leer_gkn_a_dataframe(path_archivo, fecha_hora, hole_no, project, probe_no):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    idx_tabla = None
    for idx, line in enumerate(lines):
        if line.strip().startswith("FLEVEL"):
            idx_tabla = idx
            break
    if idx_tabla is not None:
        # Preprocesar: eliminar comas de cada línea de la tabla antes de leer con pandas
        tabla_limpia = []
        for line in lines[idx_tabla:]:
            tabla_limpia.append(line.replace(",", ""))
        from io import StringIO
        tabla_str = "".join(tabla_limpia)
        df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
        # Crear columna de número de lectura (comienza en 1 para cada archivo)
        df.insert(0, 'inclinometro', hole_no)
        df.insert(1, 'numero_lectura', range(1, len(df) + 1))
        df.insert(2, 'fecha_hora', fecha_hora)
        df['project'] = project
        df['probe_no'] = probe_no
        return df
    return None

for archivo in archivos_gkn:
    fecha_hora, hole_no, project, probe_no = extraer_metadatos_gkn(archivo)
    df = leer_gkn_a_dataframe(archivo, fecha_hora, hole_no, project, probe_no)
    if df is not None:
        dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)
output_csv = '/content/lecturas_inclinometro_unificado.csv'
df_total.to_csv(output_csv, index=False)

print(f"Archivo CSV guardado en: {output_csv}")
df_total.head()

Archivo CSV guardado en: /content/lecturas_inclinometro_unificado.csv


<ipython-input-61-0a22433806e0>:50: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-61-0a22433806e0>:50: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-61-0a22433806e0>:50: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-61-0a22433806e0>:50: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_wh

,inclinometro,numero_lectura,fecha_hora,FLEVEL,A+,A-,B+,B-,project,probe_no
0,MD-IN1-150,1,2022-06-29 10:44:28,55.5,970,-994,-14,105,MD,Probe-93
1,MD-IN1-150,2,2022-06-29 10:44:28,55.0,968,-983,-15,110,MD,Probe-93
2,MD-IN1-150,3,2022-06-29 10:44:28,54.5,960,-976,-14,110,MD,Probe-93
3,MD-IN1-150,4,2022-06-29 10:44:28,54.0,975,-992,-7,98,MD,Probe-93
4,MD-IN1-150,5,2022-06-29 10:44:28,53.5,982,-996,5,90,MD,Probe-93


In [1]:
import pandas as pd
import glob
import os
from datetime import datetime

carpeta_lecturas = '/content/'
archivos_gkn = sorted(glob.glob(os.path.join(carpeta_lecturas, '*.gkn')))
dfs = []

def extraer_metadatos_gkn(path_archivo):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    fecha, hora, hole_no, project, probe_no = None, None, None, None, None
    for line in lines:
        if line.startswith("DATE"):
            fecha_str = line.split(":", 1)[1].strip()
            fecha = datetime.strptime(fecha_str, "%d/%m/%y")
        elif line.startswith("TIME"):
            hora_str = line.split(":", 1)[1].strip()
            hora = datetime.strptime(hora_str, "%H:%M:%S").time()
        elif line.startswith("HOLE NO"):
            hole_no = line.split(":", 1)[1].strip()
        elif line.startswith("PROJECT"):
            project = line.split(":", 1)[1].strip()
        elif line.startswith("PROBE NO"):
            probe_no = line.split(":", 1)[1].strip()
    if fecha and hora:
        fecha_hora = datetime.combine(fecha.date(), hora)
    elif fecha:
        fecha_hora = fecha
    else:
        fecha_hora = None
    return fecha_hora, hole_no, project, probe_no

def leer_gkn_a_dataframe(path_archivo, fecha_hora, hole_no, project, probe_no):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    idx_tabla = None
    for idx, line in enumerate(lines):
        if line.strip().startswith("FLEVEL"):
            idx_tabla = idx
            break
    if idx_tabla is not None:
        tabla_limpia = []
        for line in lines[idx_tabla:]:
            tabla_limpia.append(line.replace(",", ""))
        from io import StringIO
        tabla_str = "".join(tabla_limpia)
        df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
        df.insert(0, 'inclinometro', hole_no)
        df.insert(1, 'Punto de Lectura', range(1, len(df) + 1))
        df.insert(2, 'fecha_hora', fecha_hora)
        df['project'] = project
        df['probe_no'] = probe_no
        return df
    return None

for archivo in archivos_gkn:
    fecha_hora, hole_no, project, probe_no = extraer_metadatos_gkn(archivo)
    df = leer_gkn_a_dataframe(archivo, fecha_hora, hole_no, project, probe_no)
    if df is not None:
        dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)

# Cambia el nombre de n_lectura a Nro. Lectura y la agrega después de Punto de Lectura
df_total['Nro. Lectura'] = df_total.groupby('fecha_hora').cumcount() + 1
cols = df_total.columns.tolist()
# Mueve 'Nro. Lectura' después de 'Punto de Lectura'
cols.insert(3, cols.pop(cols.index('Nro. Lectura')))
df_total = df_total[cols]

output_csv = '/content/lecturas_inclinometro_unificado.csv'
df_total.to_csv(output_csv, index=False)
print(f"Archivo CSV guardado en: {output_csv}")
df_total.head()

<ipython-input-1-6c82479d3ec8>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-1-6c82479d3ec8>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-1-6c82479d3ec8>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-1-6c82479d3ec8>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whites

Archivo CSV guardado en: /content/lecturas_inclinometro_unificado.csv


,inclinometro,Punto de Lectura,fecha_hora,Nro. Lectura,FLEVEL,A+,A-,B+,B-,project,probe_no
0,MD-IN1-A1,1,2019-03-25 17:10:08,1,91.5,-130,123.0,-165,183.0,MD,p 12
1,MD-IN1-A1,2,2019-03-25 17:10:08,2,91.0,-103,96.0,-116,156.0,MD,p 12
2,MD-IN1-A1,3,2019-03-25 17:10:08,3,90.5,-77,71.0,-75,111.0,MD,p 12
3,MD-IN1-A1,4,2019-03-25 17:10:08,4,90.0,-35,34.0,-44,87.0,MD,p 12
4,MD-IN1-A1,5,2019-03-25 17:10:08,5,89.5,-34,30.0,-73,118.0,MD,p 12


In [2]:
import pandas as pd
import glob
import os
from datetime import datetime

carpeta_lecturas = '/content/'
archivos_gkn = sorted(glob.glob(os.path.join(carpeta_lecturas, '*.gkn')))
dfs = []

def extraer_metadatos_gkn(path_archivo):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    fecha, hora, hole_no, project, probe_no = None, None, None, None, None
    for line in lines:
        if line.startswith("DATE"):
            fecha_str = line.split(":", 1)[1].strip()
            fecha = datetime.strptime(fecha_str, "%d/%m/%y")
        elif line.startswith("TIME"):
            hora_str = line.split(":", 1)[1].strip()
            hora = datetime.strptime(hora_str, "%H:%M:%S").time()
        elif line.startswith("HOLE NO"):
            hole_no = line.split(":", 1)[1].strip()
        elif line.startswith("PROJECT"):
            project = line.split(":", 1)[1].strip()
        elif line.startswith("PROBE NO"):
            probe_no = line.split(":", 1)[1].strip()
    if fecha and hora:
        fecha_hora = datetime.combine(fecha.date(), hora)
    elif fecha:
        fecha_hora = fecha
    else:
        fecha_hora = None
    return fecha_hora, hole_no, project, probe_no

def leer_gkn_a_dataframe(path_archivo, fecha_hora, hole_no, project, probe_no):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    idx_tabla = None
    for idx, line in enumerate(lines):
        if line.strip().startswith("FLEVEL"):
            idx_tabla = idx
            break
    if idx_tabla is not None:
        tabla_limpia = []
        for line in lines[idx_tabla:]:
            tabla_limpia.append(line.replace(",", ""))
        from io import StringIO
        tabla_str = "".join(tabla_limpia)
        df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
        df.insert(0, 'inclinometro', hole_no)
        df.insert(1, 'Punto de Lectura', range(1, len(df) + 1))
        df.insert(2, 'fecha_hora', fecha_hora)
        df['project'] = project
        df['probe_no'] = probe_no
        return df
    return None

for archivo in archivos_gkn:
    fecha_hora, hole_no, project, probe_no = extraer_metadatos_gkn(archivo)
    df = leer_gkn_a_dataframe(archivo, fecha_hora, hole_no, project, probe_no)
    if df is not None:
        dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)

# Extraer la fecha (sin hora) para numerar correlativamente por día
df_total['fecha'] = pd.to_datetime(df_total['fecha_hora']).dt.date

# Ordenar por fecha y (opcional: por inclinometro, por hora, etc.)
df_total = df_total.sort_values(['fecha', 'fecha_hora', 'inclinometro']).reset_index(drop=True)

# Asignar Nro. Lectura correlativo dentro de cada fecha
df_total['Nro. Lectura'] = df_total.groupby('fecha').cumcount() + 1

# Opcional: reordenar columnas si lo deseas
cols = df_total.columns.tolist()
# Mueve 'Nro. Lectura' después de 'Punto de Lectura'
cols.insert(3, cols.pop(cols.index('Nro. Lectura')))
df_total = df_total[cols]

output_csv = '/content/lecturas_inclinometro_unificado.csv'
df_total.to_csv(output_csv, index=False)
print(f"Archivo CSV guardado en: {output_csv}")
df_total.head()

<ipython-input-2-98dbd1f15188>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-2-98dbd1f15188>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-2-98dbd1f15188>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-2-98dbd1f15188>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whites

Archivo CSV guardado en: /content/lecturas_inclinometro_unificado.csv


,inclinometro,Punto de Lectura,fecha_hora,Nro. Lectura,FLEVEL,A+,A-,B+,B-,project,probe_no,fecha
0,MD-IN1-A1,1,2019-03-25 17:10:08,1,91.5,-130,123.0,-165,183.0,MD,p 12,2019-03-25
1,MD-IN1-A1,2,2019-03-25 17:10:08,2,91.0,-103,96.0,-116,156.0,MD,p 12,2019-03-25
2,MD-IN1-A1,3,2019-03-25 17:10:08,3,90.5,-77,71.0,-75,111.0,MD,p 12,2019-03-25
3,MD-IN1-A1,4,2019-03-25 17:10:08,4,90.0,-35,34.0,-44,87.0,MD,p 12,2019-03-25
4,MD-IN1-A1,5,2019-03-25 17:10:08,5,89.5,-34,30.0,-73,118.0,MD,p 12,2019-03-25


In [3]:
import pandas as pd
import glob
import os
from datetime import datetime

carpeta_lecturas = '/content/'
archivos_gkn = sorted(glob.glob(os.path.join(carpeta_lecturas, '*.gkn')))
dfs = []

def extraer_metadatos_gkn(path_archivo):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    fecha, hora, hole_no, project, probe_no = None, None, None, None, None
    for line in lines:
        if line.startswith("DATE"):
            fecha_str = line.split(":", 1)[1].strip()
            fecha = datetime.strptime(fecha_str, "%d/%m/%y")
        elif line.startswith("TIME"):
            hora_str = line.split(":", 1)[1].strip()
            hora = datetime.strptime(hora_str, "%H:%M:%S").time()
        elif line.startswith("HOLE NO"):
            hole_no = line.split(":", 1)[1].strip()
        elif line.startswith("PROJECT"):
            project = line.split(":", 1)[1].strip()
        elif line.startswith("PROBE NO"):
            probe_no = line.split(":", 1)[1].strip()
    if fecha and hora:
        fecha_hora = datetime.combine(fecha.date(), hora)
    elif fecha:
        fecha_hora = fecha
    else:
        fecha_hora = None
    return fecha_hora, hole_no, project, probe_no

def leer_gkn_a_dataframe(path_archivo, fecha_hora, hole_no, project, probe_no):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    idx_tabla = None
    for idx, line in enumerate(lines):
        if line.strip().startswith("FLEVEL"):
            idx_tabla = idx
            break
    if idx_tabla is not None:
        tabla_limpia = []
        for line in lines[idx_tabla:]:
            tabla_limpia.append(line.replace(",", ""))
        from io import StringIO
        tabla_str = "".join(tabla_limpia)
        df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
        df.insert(0, 'inclinometro', hole_no)
        df['project'] = project
        df['probe_no'] = probe_no
        df.insert(1, 'fecha_hora', fecha_hora)
        df.insert(2, 'Punto de Lectura', range(1, len(df) + 1))
        return df
    return None

for archivo in archivos_gkn:
    fecha_hora, hole_no, project, probe_no = extraer_metadatos_gkn(archivo)
    df = leer_gkn_a_dataframe(archivo, fecha_hora, hole_no, project, probe_no)
    if df is not None:
        dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)

# Extraer solo la fecha (sin hora)
df_total['fecha'] = pd.to_datetime(df_total['fecha_hora']).dt.date

# Ordenar por fecha (ascendente), luego por hora y luego por inclinometro si quieres
df_total = df_total.sort_values(['fecha', 'fecha_hora', 'inclinometro']).reset_index(drop=True)

# Crear 'Lectura Nro' que se reinicia por fecha, comenzando por 1 en la más antigua
df_total['Lectura Nro'] = df_total.groupby('fecha').cumcount() + 1

# Reorganizar columnas:
# 1. inclinometro
# 2. fecha_hora
# 3. Lectura Nro
# 4. Punto de Lectura
# 5. ...otras columnas originales...
# 6. project, probe_no, etc
# 7. (opcional) eliminar la columna auxiliar 'fecha'
columnas_finales = ['inclinometro', 'fecha_hora', 'Lectura Nro', 'Punto de Lectura']
# Agrega el resto de columnas que no están en la lista anterior
resto = [c for c in df_total.columns if c not in columnas_finales + ['fecha']]
columnas_finales += resto
df_total = df_total[columnas_finales]

# (Opcional) elimina la columna auxiliar 'fecha'
if 'fecha' in df_total.columns:
    df_total = df_total.drop(columns=['fecha'])

output_csv = '/content/lecturas_inclinometro_unificado.csv'
df_total.to_csv(output_csv, index=False)
print(f"Archivo CSV guardado en: {output_csv}")
df_total.head()

<ipython-input-3-37e04a1e8c21>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-3-37e04a1e8c21>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-3-37e04a1e8c21>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-3-37e04a1e8c21>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whites

Archivo CSV guardado en: /content/lecturas_inclinometro_unificado.csv


,inclinometro,fecha_hora,Lectura Nro,Punto de Lectura,FLEVEL,A+,A-,B+,B-,project,probe_no
0,MD-IN1-A1,2019-03-25 17:10:08,1,1,91.5,-130,123.0,-165,183.0,MD,p 12
1,MD-IN1-A1,2019-03-25 17:10:08,2,2,91.0,-103,96.0,-116,156.0,MD,p 12
2,MD-IN1-A1,2019-03-25 17:10:08,3,3,90.5,-77,71.0,-75,111.0,MD,p 12
3,MD-IN1-A1,2019-03-25 17:10:08,4,4,90.0,-35,34.0,-44,87.0,MD,p 12
4,MD-IN1-A1,2019-03-25 17:10:08,5,5,89.5,-34,30.0,-73,118.0,MD,p 12


In [4]:
import pandas as pd
import glob
import os
from datetime import datetime

carpeta_lecturas = '/content/'
archivos_gkn = sorted(glob.glob(os.path.join(carpeta_lecturas, '*.gkn')))
dfs = []

def extraer_metadatos_gkn(path_archivo):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    fecha, hora, hole_no, project, probe_no = None, None, None, None, None
    for line in lines:
        if line.startswith("DATE"):
            fecha_str = line.split(":", 1)[1].strip()
            fecha = datetime.strptime(fecha_str, "%d/%m/%y")
        elif line.startswith("TIME"):
            hora_str = line.split(":", 1)[1].strip()
            hora = datetime.strptime(hora_str, "%H:%M:%S").time()
        elif line.startswith("HOLE NO"):
            hole_no = line.split(":", 1)[1].strip()
        elif line.startswith("PROJECT"):
            project = line.split(":", 1)[1].strip()
        elif line.startswith("PROBE NO"):
            probe_no = line.split(":", 1)[1].strip()
    if fecha and hora:
        fecha_hora = datetime.combine(fecha.date(), hora)
    elif fecha:
        fecha_hora = fecha
    else:
        fecha_hora = None
    return fecha_hora, hole_no, project, probe_no

def leer_gkn_a_dataframe(path_archivo, fecha_hora, hole_no, project, probe_no):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    idx_tabla = None
    for idx, line in enumerate(lines):
        if line.strip().startswith("FLEVEL"):
            idx_tabla = idx
            break
    if idx_tabla is not None:
        tabla_limpia = []
        for line in lines[idx_tabla:]:
            tabla_limpia.append(line.replace(",", ""))
        from io import StringIO
        tabla_str = "".join(tabla_limpia)
        df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
        df.insert(0, 'inclinometro', hole_no)
        df['project'] = project
        df['probe_no'] = probe_no
        df.insert(1, 'fecha_hora', fecha_hora)
        df.insert(2, 'Punto de Lectura', range(1, len(df) + 1))
        return df
    return None

for archivo in archivos_gkn:
    fecha_hora, hole_no, project, probe_no = extraer_metadatos_gkn(archivo)
    df = leer_gkn_a_dataframe(archivo, fecha_hora, hole_no, project, probe_no)
    if df is not None:
        dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)

# Extraer solo la fecha (sin hora)
df_total['solo_fecha'] = pd.to_datetime(df_total['fecha_hora']).dt.date

# Obtener las fechas únicas en orden ascendente
fechas_ordenadas = sorted(df_total['solo_fecha'].unique())

# Crear un diccionario de fecha a número de lectura correlativo
mapa_lectura_nro = {fecha: nro+1 for nro, fecha in enumerate(fechas_ordenadas)}

# Asignar el número de lectura según la fecha
df_total['Lectura Nro'] = df_total['solo_fecha'].map(mapa_lectura_nro)

# Reorganizar columnas: inclinometro, fecha_hora, Lectura Nro, Punto de Lectura, ...
columnas_finales = ['inclinometro', 'fecha_hora', 'Lectura Nro', 'Punto de Lectura']
resto = [c for c in df_total.columns if c not in columnas_finales + ['solo_fecha']]
columnas_finales += resto
df_total = df_total[columnas_finales]

# (Opcional) eliminar la columna auxiliar 'solo_fecha'
if 'solo_fecha' in df_total.columns:
    df_total = df_total.drop(columns=['solo_fecha'])

output_csv = '/content/lecturas_inclinometro_unificado.csv'
df_total.to_csv(output_csv, index=False)
print(f"Archivo CSV guardado en: {output_csv}")
df_total.head()

<ipython-input-4-6ae0d4fce31c>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-4-6ae0d4fce31c>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-4-6ae0d4fce31c>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-4-6ae0d4fce31c>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whites

Archivo CSV guardado en: /content/lecturas_inclinometro_unificado.csv


,inclinometro,fecha_hora,Lectura Nro,Punto de Lectura,FLEVEL,A+,A-,B+,B-,project,probe_no
0,MD-IN1-A1,2019-03-25 17:10:08,1,1,91.5,-130,123.0,-165,183.0,MD,p 12
1,MD-IN1-A1,2019-03-25 17:10:08,1,2,91.0,-103,96.0,-116,156.0,MD,p 12
2,MD-IN1-A1,2019-03-25 17:10:08,1,3,90.5,-77,71.0,-75,111.0,MD,p 12
3,MD-IN1-A1,2019-03-25 17:10:08,1,4,90.0,-35,34.0,-44,87.0,MD,p 12
4,MD-IN1-A1,2019-03-25 17:10:08,1,5,89.5,-34,30.0,-73,118.0,MD,p 12


In [5]:
import pandas as pd
import glob
import os
from datetime import datetime

carpeta_lecturas = '/content/'
archivos_gkn = sorted(glob.glob(os.path.join(carpeta_lecturas, '*.gkn')))
dfs = []

def extraer_metadatos_gkn(path_archivo):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    fecha, hora, hole_no, project, probe_no = None, None, None, None, None
    for line in lines:
        if line.startswith("DATE"):
            fecha_str = line.split(":", 1)[1].strip()
            fecha = datetime.strptime(fecha_str, "%d/%m/%y")
        elif line.startswith("TIME"):
            hora_str = line.split(":", 1)[1].strip()
            hora = datetime.strptime(hora_str, "%H:%M:%S").time()
        elif line.startswith("HOLE NO"):
            hole_no = line.split(":", 1)[1].strip()
        elif line.startswith("PROJECT"):
            project = line.split(":", 1)[1].strip()
        elif line.startswith("PROBE NO"):
            probe_no = line.split(":", 1)[1].strip()
    if fecha and hora:
        fecha_hora = datetime.combine(fecha.date(), hora)
    elif fecha:
        fecha_hora = fecha
    else:
        fecha_hora = None
    return fecha_hora, hole_no, project, probe_no

def leer_gkn_a_dataframe(path_archivo, fecha_hora, hole_no, project, probe_no):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    idx_tabla = None
    for idx, line in enumerate(lines):
        if line.strip().startswith("FLEVEL"):
            idx_tabla = idx
            break
    if idx_tabla is not None:
        tabla_limpia = []
        for line in lines[idx_tabla:]:
            tabla_limpia.append(line.replace(",", ""))
        from io import StringIO
        tabla_str = "".join(tabla_limpia)
        df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
        df.insert(0, 'inclinometro', hole_no)
        df['project'] = project
        df['probe_no'] = probe_no
        df.insert(1, 'fecha_hora', fecha_hora)
        df.insert(2, 'Punto de Lectura', range(1, len(df) + 1))
        return df
    return None

for archivo in archivos_gkn:
    fecha_hora, hole_no, project, probe_no = extraer_metadatos_gkn(archivo)
    df = leer_gkn_a_dataframe(archivo, fecha_hora, hole_no, project, probe_no)
    if df is not None:
        dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)

# Extraer solo la fecha (sin hora)
df_total['solo_fecha'] = pd.to_datetime(df_total['fecha_hora']).dt.date

# Obtener las fechas únicas en orden ascendente
fechas_ordenadas = sorted(df_total['solo_fecha'].unique())

# Crear un diccionario de fecha a número de lectura correlativo
mapa_lectura_nro = {fecha: nro+1 for nro, fecha in enumerate(fechas_ordenadas)}

# Asignar el número de lectura según la fecha
df_total['Lectura Nro'] = df_total['solo_fecha'].map(mapa_lectura_nro)

# Reorganizar columnas: inclinometro, fecha_hora, Lectura Nro, Punto de Lectura, ...
columnas_finales = ['inclinometro', 'fecha_hora', 'Lectura Nro', 'Punto de Lectura']
resto = [c for c in df_total.columns if c not in columnas_finales + ['solo_fecha']]
columnas_finales += resto
df_total = df_total[columnas_finales]

# Eliminar la columna auxiliar 'solo_fecha'
df_total = df_total.drop(columns=['solo_fecha'])

# Renombrar columnas según lo solicitado
df_total = df_total.rename(columns={
    'inclinometro': 'Inclinometro',
    'fecha_hora': 'Fecha - Hora',
    'Lectura Nro': 'Lectura',
    'Punto de Lectura': 'Puntos Leidos',
    'project': 'Proyecto',
    'probe_no': 'Sonda'
})

output_csv = '/content/lecturas_inclinometro_unificado.csv'
df_total.to_csv(output_csv, index=False)
print(f"Archivo CSV guardado en: {output_csv}")
df_total.head()

<ipython-input-5-e893e8b56e8b>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-5-e893e8b56e8b>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-5-e893e8b56e8b>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-5-e893e8b56e8b>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whites

KeyError: "['solo_fecha'] not found in axis"

In [7]:
import pandas as pd
import glob
import os
from datetime import datetime

carpeta_lecturas = '/content/'
archivos_gkn = sorted(glob.glob(os.path.join(carpeta_lecturas, '*.gkn')))
dfs = []

def extraer_metadatos_gkn(path_archivo):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    fecha, hora, hole_no, project, probe_no = None, None, None, None, None
    for line in lines:
        if line.startswith("DATE"):
            fecha_str = line.split(":", 1)[1].strip()
            fecha = datetime.strptime(fecha_str, "%d/%m/%y")
        elif line.startswith("TIME"):
            hora_str = line.split(":", 1)[1].strip()
            hora = datetime.strptime(hora_str, "%H:%M:%S").time()
        elif line.startswith("HOLE NO"):
            hole_no = line.split(":", 1)[1].strip()
        elif line.startswith("PROJECT"):
            project = line.split(":", 1)[1].strip()
        elif line.startswith("PROBE NO"):
            probe_no = line.split(":", 1)[1].strip()
    if fecha and hora:
        fecha_hora = datetime.combine(fecha.date(), hora)
    elif fecha:
        fecha_hora = fecha
    else:
        fecha_hora = None
    return fecha_hora, hole_no, project, probe_no

def leer_gkn_a_dataframe(path_archivo, fecha_hora, hole_no, project, probe_no):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    idx_tabla = None
    for idx, line in enumerate(lines):
        if line.strip().startswith("FLEVEL"):
            idx_tabla = idx
            break
    if idx_tabla is not None:
        tabla_limpia = []
        for line in lines[idx_tabla:]:
            tabla_limpia.append(line.replace(",", ""))
        from io import StringIO
        tabla_str = "".join(tabla_limpia)
        df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
        df.insert(0, 'inclinometro', hole_no)
        df['project'] = project
        df['probe_no'] = probe_no
        df.insert(1, 'fecha_hora', fecha_hora)
        df.insert(2, 'Punto de Lectura', range(1, len(df) + 1))
        return df
    return None

for archivo in archivos_gkn:
    fecha_hora, hole_no, project, probe_no = extraer_metadatos_gkn(archivo)
    df = leer_gkn_a_dataframe(archivo, fecha_hora, hole_no, project, probe_no)
    if df is not None:
        dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)

# Extraer solo la fecha (sin hora)
df_total['solo_fecha'] = pd.to_datetime(df_total['fecha_hora']).dt.date

# Obtener las fechas únicas en orden ascendente
fechas_ordenadas = sorted(df_total['solo_fecha'].unique())

# Crear un diccionario de fecha a número de lectura correlativo
mapa_lectura_nro = {fecha: nro+1 for nro, fecha in enumerate(fechas_ordenadas)}

# Asignar el número de lectura según la fecha
df_total['Lectura Nro'] = df_total['solo_fecha'].map(mapa_lectura_nro)

# Reorganizar columnas: inclinometro, fecha_hora, Lectura Nro, Punto de Lectura, ...
columnas_finales = ['inclinometro', 'fecha_hora', 'Lectura Nro', 'Punto de Lectura']
resto = [c for c in df_total.columns if c not in columnas_finales + ['solo_fecha']]
columnas_finales += resto
df_total = df_total[columnas_finales]

# Eliminar la columna auxiliar 'solo_fecha' (sin error si no existe)
df_total = df_total.drop(columns=['solo_fecha'], errors='ignore')

# Renombrar columnas según lo solicitado
df_total = df_total.rename(columns={
    'inclinometro': 'Inclinometro',
    'fecha_hora': 'Fecha - Hora',
    'Lectura Nro': 'Lectura',
    'Punto de Lectura': 'Puntos Leidos',
    'project': 'Proyecto',
    'probe_no': 'Sonda'
})

output_csv = '/content/lecturas_inclinometro_unificado.csv'
df_total.to_csv(output_csv, index=False)
print(f"Archivo CSV guardado en: {output_csv}")
df_total.head()

<ipython-input-7-99d00d34f7d3>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-7-99d00d34f7d3>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-7-99d00d34f7d3>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-7-99d00d34f7d3>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whites

Archivo CSV guardado en: /content/lecturas_inclinometro_unificado.csv


<ipython-input-7-99d00d34f7d3>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-7-99d00d34f7d3>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-7-99d00d34f7d3>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-7-99d00d34f7d3>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whites

,Inclinometro,Fecha - Hora,Lectura,Puntos Leidos,FLEVEL,A+,A-,B+,B-,Proyecto,Sonda
0,MD-IN1-A1,2019-03-25 17:10:08,1,1,91.5,-130,123.0,-165,183.0,MD,p 12
1,MD-IN1-A1,2019-03-25 17:10:08,1,2,91.0,-103,96.0,-116,156.0,MD,p 12
2,MD-IN1-A1,2019-03-25 17:10:08,1,3,90.5,-77,71.0,-75,111.0,MD,p 12
3,MD-IN1-A1,2019-03-25 17:10:08,1,4,90.0,-35,34.0,-44,87.0,MD,p 12
4,MD-IN1-A1,2019-03-25 17:10:08,1,5,89.5,-34,30.0,-73,118.0,MD,p 12


In [8]:
import pandas as pd
import glob
import os
from datetime import datetime

carpeta_lecturas = '/content/'
archivos_gkn = sorted(glob.glob(os.path.join(carpeta_lecturas, '*.gkn')))
dfs = []

def extraer_metadatos_gkn(path_archivo):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    fecha, hora, hole_no, project, probe_no = None, None, None, None, None
    for line in lines:
        if line.startswith("DATE"):
            fecha_str = line.split(":", 1)[1].strip()
            fecha = datetime.strptime(fecha_str, "%d/%m/%y")
        elif line.startswith("TIME"):
            hora_str = line.split(":", 1)[1].strip()
            hora = datetime.strptime(hora_str, "%H:%M:%S").time()
        elif line.startswith("HOLE NO"):
            hole_no = line.split(":", 1)[1].strip()
        elif line.startswith("PROJECT"):
            project = line.split(":", 1)[1].strip()
        elif line.startswith("PROBE NO"):
            probe_no = line.split(":", 1)[1].strip()
    if fecha and hora:
        fecha_hora = datetime.combine(fecha.date(), hora)
    elif fecha:
        fecha_hora = fecha
    else:
        fecha_hora = None
    return fecha_hora, hole_no, project, probe_no

def leer_gkn_a_dataframe(path_archivo, fecha_hora, hole_no, project, probe_no):
    with open(path_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    idx_tabla = None
    for idx, line in enumerate(lines):
        if line.strip().startswith("FLEVEL"):
            idx_tabla = idx
            break
    if idx_tabla is not None:
        tabla_limpia = []
        for line in lines[idx_tabla:]:
            tabla_limpia.append(line.replace(",", ""))
        from io import StringIO
        tabla_str = "".join(tabla_limpia)
        df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
        df.insert(0, 'inclinometro', hole_no)
        df['project'] = project
        df['probe_no'] = probe_no
        df.insert(1, 'fecha_hora', fecha_hora)
        df.insert(2, 'Punto de Lectura', range(1, len(df) + 1))
        return df
    return None

for archivo in archivos_gkn:
    fecha_hora, hole_no, project, probe_no = extraer_metadatos_gkn(archivo)
    df = leer_gkn_a_dataframe(archivo, fecha_hora, hole_no, project, probe_no)
    if df is not None:
        dfs.append(df)

if not dfs:
    print("No se encontraron archivos válidos o ningún archivo tiene datos procesables.")
else:
    df_total = pd.concat(dfs, ignore_index=True)
    # Extraer solo la fecha (sin hora)
    df_total['solo_fecha'] = pd.to_datetime(df_total['fecha_hora']).dt.date

    # Obtener las fechas únicas en orden ascendente
    fechas_ordenadas = sorted(df_total['solo_fecha'].unique())

    # Crear un diccionario de fecha a número de lectura correlativo
    mapa_lectura_nro = {fecha: nro+1 for nro, fecha in enumerate(fechas_ordenadas)}

    # Asignar el número de lectura según la fecha
    df_total['Lectura Nro'] = df_total['solo_fecha'].map(mapa_lectura_nro)

    # Reorganizar columnas: inclinometro, fecha_hora, Lectura Nro, Punto de Lectura, ...
    columnas_finales = ['inclinometro', 'fecha_hora', 'Lectura Nro', 'Punto de Lectura']
    resto = [c for c in df_total.columns if c not in columnas_finales + ['solo_fecha']]
    columnas_finales += resto
    df_total = df_total[columnas_finales]

    # Eliminar la columna auxiliar 'solo_fecha' (sin error si no existe)
    df_total = df_total.drop(columns=['solo_fecha'], errors='ignore')

    # Renombrar columnas según lo solicitado
    df_total = df_total.rename(columns={
        'inclinometro': 'Inclinometro',
        'fecha_hora': 'Fecha-Hora',        # <--- Cambio aquí
        'Lectura Nro': 'Lectura',
        'Punto de Lectura': 'Puntos Leidos',
        'project': 'Proyecto',
        'probe_no': 'Sonda'
    })

    output_csv = '/content/lecturas_inclinometro_unificado.csv'
    df_total.to_csv(output_csv, index=False)
    print(f"Archivo CSV guardado en: {output_csv}")
    display(df_total.head())

<ipython-input-8-ce06038fdb35>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-8-ce06038fdb35>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-8-ce06038fdb35>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whitespace=True, engine='python')
<ipython-input-8-ce06038fdb35>:49: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(StringIO(tabla_str), delim_whites

Archivo CSV guardado en: /content/lecturas_inclinometro_unificado.csv


,Inclinometro,Fecha-Hora,Lectura,Puntos Leidos,FLEVEL,A+,A-,B+,B-,Proyecto,Sonda
0,MD-IN1-A1,2019-03-25 17:10:08,1,1,91.5,-130,123.0,-165,183.0,MD,p 12
1,MD-IN1-A1,2019-03-25 17:10:08,1,2,91.0,-103,96.0,-116,156.0,MD,p 12
2,MD-IN1-A1,2019-03-25 17:10:08,1,3,90.5,-77,71.0,-75,111.0,MD,p 12
3,MD-IN1-A1,2019-03-25 17:10:08,1,4,90.0,-35,34.0,-44,87.0,MD,p 12
4,MD-IN1-A1,2019-03-25 17:10:08,1,5,89.5,-34,30.0,-73,118.0,MD,p 12
